# Section 25: Complete Data Science Project with Multiple Linear Regression

## Objectives
- Repeat the Mod 1 Project analysis in a streamlined way.

- Learn about `pandas_profiling` for quick and easy EDA (don't use on Mod Projects please).

- Intro to the idea of pipelines / programmatic construction of models

- Learn about [Variance Inflation Factor](https://etav.github.io/python/vif_factor_python.html) and how to use it to address multicollinearity. ( [Wikipedia: VIF](https://en.wikipedia.org/wiki/Variance_inflation_factor))




### Questions?
- 

**Work Tracker (How long did the project take me?):**
- Start: 12/10/19 ~8:45pm 
- End: 12/10/19 11:00 pm

# OBTAIN

In [1]:
!pip install -U fsds_100719
from fsds_100719.imports import *

fsds_1007219  v0.4.45 loaded.  Read the docs: https://fsds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds_100719,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


In [ ]:
import scipy.stats as stats
import statsmodels.api as sms
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import pandas_profiling

In [3]:
## Set Display options
pd.set_option('display.precision',3)
pd.set_option('display.max_columns',0)

## Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = fs.datasets.load_mod1_proj()
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,1955,0.0,98178,47.511,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.721,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,1933,NaN,98028,47.738,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.521,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.617,-122.045,1800,7503


# SCRUB / EXPLORE

In [5]:
drop_cols = ['id','date','view']
df.drop(drop_cols,axis=1,inplace=True)

In [6]:
pandas_profiling.ProfileReport(df)

In [ ]:
## Replace Values
replace_data = {'sqft_basement':('?',"0.0")}

for col,replace in replace_data.items():
    df[col] = df[col].replace(replace[0],replace[1])

In [ ]:
## Null Values
fill_data = {'waterfront':0,
            'yr_renovated':0}

for col,val in fill_data.items():
    df[col] = df[col].fillna(val)

In [ ]:
## Recast Columns
recast_cols = {'sqft_basement':'float',
              'waterfront':'int'}

for col,dtype in recast_cols.items():
    df[col] = df[col].astype(dtype)

In [ ]:
# pandas_profiling.ProfileReport(df)

In [ ]:
# features = '+'.join(df.drop(target,axis=1).columns)
# col ='bedrooms'
# features = features.replace(col,f"C({col})")
# 'price~'+features
cols_to_use= list(df.columns)
exclude_cols=['']
[cols_to_use.remove(ecol) for ecol in exclude_cols if ecol in cols_to_use]
cols_to_use

In [ ]:
cols= list(df.columns)
cols.remove('price')
cols

# Model 

In [ ]:
import statsmodels.api as sms
import statsmodels.formula.api as smf


def make_ols_f(df,target = 'price',cat_cols=[], exlude_cols=[], cols_to_use=None, display_summary=False ):
    import statsmodels.api as sms
    import statsmodels.formula.api as smf
    """Create and fit a statsmodels formula api ols model.
    
    Args
        df (Frame): data for ols
        target (str): name of target column
        cat_cols (list of str): columns to treat as categories (i.e. "target+C(col)")
        cols_to_use (list of str): 
    
    """
    
    if isinstance(cat_cols,str):
        cat_cols=[cat_cols]
        
    if cols_to_use is None:
        cols_to_use=list(df.drop(target,axis=1).columns)
        
    ## Remove exclude cols    
    [cols_to_use.remove(ecol) for ecol in exclude_cols if ecol in cols_to_use]

    ## Create Feature List
    features = '+'.join(cols_to_use)
    
    ## Make cat_cols into categories
    for col in cat_cols:
        features = features.replace(col,f"C({col})")
            
    ## Construct formula 
    formula = target+'~'+features
    
    model = smf.ols(formula=formula,data=df).fit()
    
    if display_summary:
        display(model.summary())
    
    return model


model = make_ols_f(df,'price',['bedrooms','zipcode'])
# smf.ols()
model.summary()

In [ ]:
import scipy.stats as stats
def diagnose_model(model):
    fig,ax= plt.subplots(ncols=2,figsize=(10,5))
    resid = model.resid 
    sms.qqplot(data=resid, dist=stats.distributions.norm,fit=True,
              line='45',ax=ax[0])
    
    ax[1].scatter(x=np.linspace(0,1,len(resid)),y=resid)
    plt.tight_layout()
    return fig, ax
    
diagnose_model(model)

# Interpret

- How does our model look? 
- What haven't we addressed yet?

# Revised Model 

## Scrub 2: Outlier Removal

In [ ]:
def find_outliers(col):
    """Use scipy to calcualte absoliute Z-scores 
    and return boolean series where True indicates it is an outlier
    Args:
        col (Series): a series/column from your DataFrame
    Returns:
        idx_outliers (Series): series of  True/False for each row in col
        
    Ex:
    >> idx_outs = find_outliers(df['bedrooms'])
    >> df_clean = df.loc[idx_outs==False]"""
    from scipy import stats
    z = np.abs(stats.zscore(col))
    idx_outliers = np.where(z>3,True,False)
    return idx_outliers


In [ ]:


def find_outliers_Z(df,col):
    """Use scipy to calcualte absoliute Z-scores 
    and return boolean series where True indicates it is an outlier
    Args:
        col (Series): a series/column from your DataFrame
    Returns:
        idx_outliers (Series): series of  True/False for each row in col
        
    Ex:
    >> idx_outs = find_outliers(df['bedrooms'])
    >> df_clean = df.loc[idx_outs==False]"""
    from scipy import stats

    col = df[col]
    z = np.abs(stats.zscore(col))
    idx_outliers = np.where(z>3,True,False)
    return idx_outliers



In [ ]:
def find_outliers_IQR(df,col):
    """Use Pandas.describe() to calculate outleirs according to Tukey's method.
    AKA Interquartile Range Removal.
    
    Args:
        df (Frame): dataframe with data
        col (str): column in df to test
        
    Returns:
        idx_outliers (Series): series of  True/False for each row in col
    """

    res = df[col].describe()

    IQR = res['75%'] - res['25%']
    lower_limit = res['25%']-(IQR*1.5)
    upper_limit = res['75%']+(IQR*1.5)


    idx_outliers = (df[col]<upper_limit) & (df[col]>lower_limit)
    return ~idx_outliers


In [ ]:
## Test outlier removal differences
outliers_z = find_outliers_Z(df,'bedrooms')
outliers_iqr = find_outliers_IQR(df,'bedrooms')

print(len(df))
outliers_z.sum(),outliers_iqr.sum()

In [ ]:
col = 'bedrooms'

axZ =df.loc[outliers_z==False].plot(kind='scatter',x=col,y='price')
axZ.set_title('Z Score')

axIQR = df.loc[outliers_iqr==False].plot(kind='scatter',x=col,y='price')
axIQR.set_title('IQR')

In [ ]:
## Identify and remove outliers
df_outliers_Z = pd.DataFrame()
df_outliers_IQR = pd.DataFrame()

for col in df.columns: #['bedrooms','price']:
    
    df_outliers_Z[col] = find_outliers_Z(df,col)
    df_outliers_IQR[col] = find_outliers_IQR(df,col)
    
idx_outs_Z = df_outliers_Z.any(axis=1)
idx_outs_IQR = df_outliers_IQR.any(axis=1)

print(idx_outs_Z.sum()*100/len(idx_outs_Z),idx_outs_IQR.sum()/len(idx_outs_IQR)*100)

In [ ]:
df.loc[idx_outs_Z==False]

In [ ]:
display(df.loc[idx_outs_Z==True].describe())
# display(df.loc[idx_outs_IQR==False].describe())

In [ ]:
df = df.loc[idx_outs_Z==False]
df.describe()

In [ ]:
## Test out Revised Model
model = make_ols_f(df,'price',['grade','zipcode'],display_summary=True)

diagnose_model(model)


In [ ]:
df

In [ ]:
# R2 = model.rsquared

# VIF = 1 / (1 - R2)
# VIF

## Explore 2: Multicollinearity with VIF

> $\large V.I.F. = \frac{1}{(1 - R^2)} $<br>
- [VIF (Variance Inflation Factor)](https://etav.github.io/python/vif_factor_python.html) 
- [Wikipedia: VIF](https://en.wikipedia.org/wiki/Variance_inflation_factor)

In [ ]:
cat_cols = ['zipcode']
model_target='price'

def vif_ols(df,exclude_col = None, cat_cols = ['grade','zipcode']):
    # let's check each column, build a model and get the r2
    vif_scores = [['Column','VIF','R2']]

    if exclude_col is not None:
        df = df.drop(exclude_col,axis=1)
        
    for column in df.columns:
        columns_to_use = df.drop(columns=[column]).columns
        target = column
        linreg = make_ols_f(df, target=target, cat_cols=cat_cols,cols_to_use=columns_to_use)
        R2 = linreg.rsquared
        VIF = 1 / (1 - R2)
    #     print(f"VIF for {column} = {VIF}")
        vif_scores.append([column, VIF, R2])

    res = fs.ds.list2df(vif_scores,index_col='Column')
    res.sort_values('R2',ascending=False,inplace=True)
    
    return res

res = vif_ols(df,exclude_col='price',)
res

In [ ]:
res['use'] = res['VIF']<5
res

In [ ]:
res[res['use']]

In [ ]:
df_use = df[res[res['use']].index]
df_use.info()

In [ ]:
res2 = vif_ols(df_use)
res2

In [ ]:
df_use['price'] = df['price'].copy()

In [ ]:
model = make_ols_f(df_use,cat_cols=['grade','zipcode'])
display(model.summary())
diagnose_model(model)

In [ ]:
model2 = make_ols_f(df,target='price',cat_cols=['grade','zipcode'],exlude_cols=['floors'])
display(model2.summary())
diagnose_model(model2)


In [ ]:
## train test spit
cat_cols = ['grade','zipcode']
exclude_cols=['floors']
target = 'price'
exclude_cols.append(target)
print(exclude_cols)
from sklearn.model_selection import train_test_split
X = df.drop(exclude_cols,axis=1)
y= df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

df_train = pd.DataFrame(X_train, columns=X.columns)
df_train[target] = y_train

df_test = pd.DataFrame(X_test, columns=X.columns)
df_test[target] = y_test

In [ ]:
model3 = make_ols_f(df_use,target,cat_cols,exclude_cols)
model3.summary()

In [ ]:
from sklearn.metrics import r2_score
r2_score(df_train['price'],model3.predict(df_train))
# r2_score()

In [ ]:
def validate_model(model, df_train, df_test):
    from sklearn.metrics import r2_score
    
    res=[['Data','R2']]
    y_hat_train = model.predict(df_train)
    y_hat_test = model.predict(df_test)
    
    res.append(['Train',r2_score(df_train['price'],y_hat_train)])
    res.append(['Test',r2_score(df_test['price'],y_hat_test)])
    
    return fs.ds.list2df(res)
        
    
validate_model(model3,df_train,df_test)    